# Setup

## Imports & `.env` Setup

In [72]:
import pandas as pd, numpy as np, tweepy as tw, streamlit as st
import requests, os, dotenv, json
from hashlib import *
dotenv.load_dotenv();

## Load Secret Variables

### Twitter

In [2]:
def do_tweepy_auth():
    key, secret_key = (os.getenv(i) for i in ["TWEET_KEY", "TWEET_SECRET_KEY"])
    auth = tw.OAuthHandler(key, secret_key)
    api = tw.API(auth, wait_on_rate_limit=True)
    return api

In [3]:
import tweepy as tw
api = do_tweepy_auth()

### Airtable

In [4]:
token, base, url = (os.getenv(i) for i in ["AIRTABLE_TOKEN",
                                           "AIRTABLE_BASE_ID",
                                           "AIRTABLE_URL"])

## Functions

### Twitter

In [5]:
def getTweets(search, method = "search_tweets", n = 50, frame = False):
    tweets = tw.Cursor(eval(f"api.{method}"),
              q=search,
              lang="en").items(n)
    tweets = list(tweets)
    print("Total Tweets fetched:", len(tweets))
    if frame:
        return pd.DataFrame([{"text":tweet._json["text"], "user":tweet.user._json["screen_name"]} for tweet in tweets])
    return tweets

In [ ]:
def getUser(username):
    return api.search_users(username, count = 1)

In [ ]:
def getTimeline(username, n = 100000):
    tweets = api.search_tweets(username, count = n)
    cleanTweets = [tweet._json["text"] for tweet in tweets if tweet.user._json["screen_name"] == username]
    return cleanTweets

## Airtable

In [6]:
def post(rater, rated, rating, table = "ratings"):
    # data = { "records": [{"fields": {"rater": "dfhn40sgqwn239045823","rated": "joeyDiaz","rating": "DEM"}}]}
    """Add data to the Airtable."""
    data = {"records": [
                {"fields": {"rater": rater, 
                            "rated": rated, 
                            "rating": rating}
                }
         ]
    }
    sheet = f"{url}/{table}"
    headers = {'Authorization': f'Bearer {token}', 
               'Content-Type': 'application/json'}
    response = requests.request("POST", sheet, headers=headers, data=json.dumps(data))
    return response

In [7]:
def get(query = None, table = "tweets"):
    """Retrieve Tweet Data"""
    sheet = f"{url}/{table}"
    headers = {'Authorization': f'Bearer {token}', 
               'Content-Type': 'application/json'}
    response = requests.request("GET", sheet, headers=headers)
    data = [{"id" : record["id"]} | record["fields"] for record in eval(response.text)["records"]]
    frame = pd.DataFrame(data)
    return frame

In [50]:
def update(username, table = "tweets", step = 1):
    """Update specific field values for a record."""
    sheet = f"{url}/{table}"
    headers = {'Authorization': f'Bearer {token}',
                'Content-Type': 'application/json'}
    record = dict(get().query(f"user == '{username}'").iloc[0, :])
    
    new_record = {"records": [ {"id": record["id"],
                                "fields": {"ratings": int(record["Rated"]) + step}
                               }
                             ]
                 }
    response = requests.request("PATCH", 
                                sheet, 
                                headers=headers, 
                                data=json.dumps(new_record))
    return response

## Hashing

In [73]:
def hash(obj):
    return sha1(obj.encode("utf-8")).hexdigest()

# Work

In [55]:
l = pd.read_csv("tweets.csv").drop(columns = "Unnamed: 0")
l.to_csv("tweets.csv", index = 0)

'4ae7c3b6ac0beff671efa8cf57386151c06e58ca53a78d83f36107316cec125f'